In [126]:
from selenium import webdriver
import time
import pandas as pd
import numpy as np

In [171]:
url = 'https://www.linkedin.com/jobs/search?keywords=Data%20Scientist&location=Russia&geoId=101728296&trk=public_jobs_jobs-search-bar_search-submit&position=1&pageNum=0'
#url = 'https://www.linkedin.com/jobs/search?keywords=Machine%20Learning&location=Russia&position=1&pageNum=0'
#url = 'https://www.linkedin.com/jobs/search?keywords=Artificial%20Intelligence&location=Russia&position=1&pageNum=0'

In [172]:
wd = webdriver.Chrome(executable_path='./chromedriver.exe')
wd.get(url)

In [173]:
no_of_jobs = int(wd.find_element_by_css_selector('h1>span').get_attribute('innerText'))

In [174]:
print(no_of_jobs)

104


In [175]:
i = 2
while i <= int(no_of_jobs/25)+1: 
    wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    i = i + 1
    try:
        wd.find_element_by_xpath('/html/body/div/div/main/section[2]/button').click()
        time.sleep(10)
    except:
        pass
        time.sleep(10)

In [176]:
job_lists = wd.find_element_by_class_name('jobs-search__results-list')
jobs = job_lists.find_elements_by_tag_name('li') # return a list

In [178]:
job_id= []
job_title = []
company_name = []
location = []
date = []
job_link = []
for job in jobs:
    job_id0 = job.get_attribute('data-id')
    job_id.append(job_id0)
 
    job_title0 = job.find_element_by_css_selector('h3').get_attribute('innerText')
    job_title.append(job_title0)
 
    company_name0 = job.find_element_by_css_selector('h4').get_attribute('innerText')
    company_name.append(company_name0)
 
    location0 = job.find_element_by_css_selector('[class="job-search-card__location"]').get_attribute('innerText')
    location.append(location0)
 
    date0 = job.find_element_by_css_selector('div>div>time').get_attribute('datetime')
    date.append(date0)
 
    job_link0 = job.find_element_by_css_selector('a').get_attribute('href')
    job_link.append(job_link0)

In [180]:
jd = []
seniority = []
emp_type = []
job_func = []
industries = []
for item in range(len(jobs)):
    job_func0=[]
    industries0=[]
    # clicking job to view job details
    job_click_path = f'/html/body/div/div/main/section[2]/ul/li[{item+1}]/div/a'
    try:
        job_click = job.find_element_by_xpath(job_click_path).click()
    except:
        print("Something went wrong")
    finally:
        time.sleep(10)
 
    jd_path = '/html/body/div/div/section/div[2]/div/section/div/div[2]/section/div'
    try:
        jd0 = job.find_element_by_xpath(jd_path).get_attribute('innerText')       
    except:
        jd.append(' ')
    else:
        jd.append(jd0)
 
    seniority_path = '/html/body/div/div/section/div[2]/div/section/div/ul/li[1]/span'
    try: 
        seniority0 = job.find_element_by_xpath(seniority_path).get_attribute('innerText')
    except:
        seniority.append(' ')
    else:
        seniority.append(seniority0)
 
    emp_type_path = '/html/body/div/div/section/div[2]/div/section/div/ul/li[2]/span'
    try: 
        emp_type0 = job.find_element_by_xpath(emp_type_path).get_attribute('innerText')
    except:
        emp_type.append(' ')
    else: 
        emp_type.append(emp_type0)

    job_func_path = '/html/body/div/div/section/div[2]/div/section/div/ul/li[3]/span'
    try: 
        job_func_elements = job.find_elements_by_xpath(job_func_path)
    except:
        job_func.append("")
    else:
        for element in job_func_elements:
            job_func0.append(element.get_attribute('innerText'))
            job_func_final = ', '.join(job_func0)
        job_func.append(job_func_final)
 
    industries_path = '/html/body/div/div/section/div[2]/div/section/div/ul/li[4]/span'
    try:
        industries_elements = job.find_elements_by_xpath(industries_path)
    except:
        industries.append(' ')
    else:
        for element in industries_elements:
            industries0.append(element.get_attribute('innerText'))
            industries_final = ', '.join(industries0)
        industries.append(industries_final)


In [182]:
job_data = pd.DataFrame({'ID': job_id,
'Date': date,
'Company': company_name,
'Title': job_title,
'Location': location,
'Description': jd,
'Level': seniority,
'Type': emp_type,
'Function': job_func,
'Industry': industries,
'Link': job_link
})
# cleaning description column
job_data['Description'] = job_data['Description'].str.replace('\n',' ')
job_data.to_csv('LinkedIn Job Data_Data Scientist.csv', encoding='utf-8', index = False)
#job_data.to_csv('LinkedIn Job Data_AI.csv', encoding='utf-8', index = False)